<a href="https://colab.research.google.com/github/cedro3/data-efficient-gans/blob/master/DiffAugment_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data-Efficient GANs with DiffAugment


## セットアップ



In [ ]:
# tensorflow 1.15.0　のインストール
!pip uninstall -y tensorflow tensorflow-probability
!pip install tensorflow-gpu==1.15.0 tensorflow-datasets==2.1.0

# Githubからコードをコピー
!git clone https://github.com/cedro3/data-efficient-gans.git
%cd data-efficient-gans/DiffAugment-stylegan2

# gdownアップデート
! pip install --upgrade gdown

# numpy 1.19.4のインストール
! pip install numpy==1.19.4 

In [ ]:
# Google drive から新垣結衣の画像と学習済みの重みをダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1rlmCCuLH4euzIwzqlYh8LiF5iizh5UWV', 'yui.zip', quiet=False)
! unzip yui.zip

# 関数の定義

In [ ]:
# 関数の定義
import tensorflow as tf
import os
import numpy as np
import PIL
import IPython
from multiprocessing import Pool
import matplotlib.pyplot as plt

from dnnlib import tflib, EasyDict
from training import misc, dataset_tool
from metrics import metric_base
from metrics.metric_defaults import metric_defaults

def _generate(network_name, num_rows, num_cols, seed, resolution):
  if seed is not None:
    np.random.seed(seed)
  with tf.Session():
    _, _, Gs = misc.load_pkl(network_name)
    z = np.random.randn(num_rows * num_cols, Gs.input_shape[1])
    outputs = Gs.run(z, None, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    outputs = np.reshape(outputs, [num_rows, num_cols, *outputs.shape[1:]])
    outputs = np.concatenate(outputs, axis=1)
    outputs = np.concatenate(outputs, axis=1)
    img = PIL.Image.fromarray(outputs)
    img = img.resize((resolution * num_cols, resolution * num_rows), PIL.Image.ANTIALIAS)
  return img

def generate(network_name, num_rows, num_cols, seed=None, resolution=128):
  with Pool(1) as pool:
    return pool.apply(_generate, (network_name, num_rows, num_cols, seed, resolution))

# データセットの作成
64×64の画像100枚からtfrecords形式のデータセットを作成します。\

In [ ]:
# 100-shot-gakki を読み込み学習用データセットを作成
data_dir = dataset_tool.create_dataset('100-shot-gakki')
training_images = []
for fname in os.listdir(data_dir):
  if fname.endswith('.jpg'):
    training_images.append(np.array(PIL.Image.open(os.path.join(data_dir, fname))))
imgs = np.reshape(training_images, [5, 20, *training_images[0].shape])
imgs = np.concatenate(imgs, axis=1)
imgs = np.concatenate(imgs, axis=1)
PIL.Image.fromarray(imgs).resize((1000, 250), PIL.Image.ANTIALIAS)

# 学習の実行
学習時間は、割り当てられているGPUによって異なります。下記を参考にして下さい。\ 
P100 : 7.3H (kimg=300)\
V100 : 4.1H (kimg=300)\
\
※割り当てがK80であったり、学習に時間を掛けたくない方は、kimg=500で学習した重みがありますので、ここはパスでOKです。

In [ ]:
# GPUの確認
!nvidia-smi

In [ ]:
# 学習の実行
!python3 run_few_shot.py --dataset=100-shot-gakki --resolution=64 --total-kimg=300

# 学習済みの重みを使う
実際に学習を行った場合は、resultsフォルダーの1段下に重み( network-snapshot-XXXXXX.pkl)が作成されますので、それをDiffAugment-stylegan2のディレクトリーに移動して下さい。\
そして、**generate() , generate_gif.py** の引数をそのファイル名に変更して下さい。\
\

In [ ]:
# 学習済みの重みを使って画像生成
generate('network-snapshot-gakki-000500.pkl', num_rows=2, num_cols=5, seed=3)

In [ ]:
# 学習済みの重みを使って、GIF動画 (interp.gif) を作成
!python3 generate_gif.py -r network-snapshot-gakki-000500.pkl -o interp.gif --num-rows=2 --num-cols=3 --seed=1
IPython.display.Image(open('interp.gif', 'rb').read())